In [1]:
# pyspark --conf spark.executor.extraClassPath=mysql-connector-java-8.0.17.jar --driver-class-path mysql-connector-java-8.0.17.jar --jars mysql-connector-java-8.0.17.jar
import os
from pyspark import SparkContext
from pyspark.sql import SparkSession

os.environ["PYSPARK_PYTHON"]="python3"
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars file:///home/mayfly/workstation/mycode/spark_study/mysql-connector-java-8.0.17.jar pyspark-shell'
# os.environ["PYSPARK_DRIVER_PYTHON"]="python3"
# os.environ['SPARK_HOME'] = "/home/mayfly/workstation/env/spark"

In [14]:
spark=SparkSession.builder.getOrCreate()
password = "password"
jdbcDF = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/spark")\
    .option("driver","com.mysql.jdbc.Driver").option("dbtable", "student")\
    .option("user", "root").option("password", password).load()
jdbcDF.show()

+---+--------+------+---+
| id|    name|gender|age|
+---+--------+------+---+
|  1| Xueqian|     F| 23|
|  2|Weiliang|     M| 24|
+---+--------+------+---+



In [15]:
from pyspark.sql.types import Row
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
studentRDD = spark.sparkContext.parallelize(["3 Rongcheng M 26","4 Guanhua M 27"]).map(lambda line : line.split(" "))
# 下面要设置模式信息
schema = StructType([StructField("name", StringType(), True),StructField("gender", StringType(), True),StructField("age",IntegerType(), True)])
rowRDD = studentRDD.map(lambda p : Row(p[1].strip(), p[2].strip(),int(p[3])))
# 建立起Row对象和模式之间的对应关系，也就是把数据和模式对应起来
studentDF = spark.createDataFrame(rowRDD, schema)
prop = {}
prop['user'] = 'root'
prop['password'] = password
prop['driver'] = "com.mysql.jdbc.Driver"
studentDF.write.jdbc("jdbc:mysql://localhost:3306/spark",'student','append', prop)

In [5]:
from pyspark.sql.types import Row
def f(x):
    rel = {}
    rel['name'] = x[0]
    rel['age'] = x[1]
    return rel

spark=SparkSession.builder.getOrCreate()
# sc = SparkContext(appName="make_rdd", master="local")
sc = spark.sparkContext
peopleDF = sc.textFile("file:///home/mayfly/workstation/mycode/spark_study/people.txt").map(lambda line : line.split(',')).map(lambda x: Row(**f(x))).toDF()
peopleDF.createOrReplaceTempView("people")  #必须注册为临时表才能供下面的查询使用 
personsDF = spark.sql("select * from people")
# personsDF.rdd.map(lambda t : "Name:"+t[0]+","+"Age:"+t[1]).foreach(print)
for t in personsDF.rdd.map(lambda t : "Name:"+t[0]+","+"Age:"+t[1]).collect():
    print(t)

Name: 29,Age:Michael
Name: 30,Age:Andy
Name: 19,Age:Justin


In [6]:
from pyspark.sql.types import Row
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType

# sc = SparkContext(appName="make_rdd", master="local")
spark=SparkSession.builder.getOrCreate()
sc = spark.sparkContext
# 生成 RDD
peopleRDD = sc.textFile("file:///home/mayfly/workstation/mycode/spark_study/people.txt")

# 定义一个模式字符串
schemaString = "name age"
 
# 根据模式字符串生成模式
fields = list(map( lambda fieldName : StructField(fieldName, StringType(), nullable = True), schemaString.split(" ")))
print(fields)
schema = StructType(fields)
print(schema)
# 从上面信息可以看出，schema描述了模式信息，模式中包含name和age两个字段
 
 
rowRDD = peopleRDD.map(lambda line : line.split(','))
print(rowRDD.collect())

rowRDD = rowRDD.map(lambda attributes : Row(attributes[0], attributes[1]))
print(rowRDD.collect())
 
peopleDF = spark.createDataFrame(rowRDD, schema)

# 必须注册为临时表才能供下面查询使用
peopleDF.createOrReplaceTempView("people")
 
results = spark.sql("SELECT * FROM people")
# results.rdd.saveAsTextFile("file:///home/mayfly/workstation/mycode/spark_study/newpeople.txt")
results = results.rdd.map( lambda attributes : "name: " + attributes[0]+","+"age:"+attributes[1])
for t in results.collect():
    print(t)

[StructField(name,StringType,true), StructField(age,StringType,true)]
StructType(List(StructField(name,StringType,true),StructField(age,StringType,true)))
[['Michael', ' 29'], ['Andy', ' 30'], ['Justin', ' 19']]
[<Row(Michael,  29)>, <Row(Andy,  30)>, <Row(Justin,  19)>]
name: Michael,age: 29
name: Andy,age: 30
name: Justin,age: 19


In [7]:
spark=SparkSession.builder.getOrCreate()
df = spark.read.json("file:///home/mayfly/workstation/mycode/spark_study/people.json")
print(df.show())

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

None


In [8]:
spark.stop()